In [1]:
#packages we need
import geopandas as gpd
%matplotlib inline  
from shapely.geometry import Point
import numpy as np
import pandas as pd
import networkx as nx
from geopy.distance import distance
from geopy.geocoders import Nominatim

In [2]:
#london geographic dataset we need
london = gpd.read_file("data/London_Ward.shp")

In [3]:
#tube geographic data we need
station=pd.read_csv('station.csv')
station=station.drop_duplicates(subset='Name')

# Read in the full zone 1 csv
zone1=pd.read_csv('zone1.csv')
#Read it into a network
tubeZone1=nx.from_pandas_dataframe(zone1,'station_name1','station_name2', create_using=nx.Graph())

In [4]:
#match network data with location data
#read station names seperately
df1=zone1['station_name1']
df2=zone1['station_name2']

#concat two columns into one
df3=pd.concat([df1,df2], axis=0)

#choose the unique names
df4=df3.unique()

#turn it into dataframe format
data = {'Name':df4}
frame = pd.DataFrame(data)

#change the station's name from Edgware Road (B) to Edgware Road (Bakerloo Line)
frame['Name'][2]='Edgware Road (Bakerloo Line)'

In [5]:
frame.head()

,Name
0,Baker Street
1,Charing Cross
2,Edgware Road (Bakerloo Line)
3,Embankment
4,Lambeth North


In [6]:
#merge location data with network data on common column 'name'
frame1=pd.merge(frame, station, on='Name')

In [7]:
#kcl geographic data
geolocator = Nominatim()
location = geolocator.geocode("WC2R 2LS")
print location

WC2R 2LS, United Kingdom


In [8]:
kcl_coords=(location.longitude, location.latitude)
kings_loc = gpd.GeoDataFrame([{"name": "KCL","geometry": Point(kcl_coords)}],crs={'init' :'epsg:4326'})
kings_loc = kings_loc.to_crs(london.crs)

In [9]:
#the buffer we use is to limit the area of searching the stations within 500 meters.
kings_buffer = kings_loc.copy() # copy the GeoDataFrame
# create a buffer of 500 meters to convert the POINTs to POLYGONs 
#   in the 'geometry' column
kings_buffer['geometry'] = kings_buffer.buffer(500)

In [10]:
#get the individual column from location-network dataset (frame1), and turn it into geodataframe
name=[frame1['Name'][i] for i in range(len(frame1))]
lon=[float(frame1['Longitude'][i]) for i in range(len(frame1))]
lat=[float(frame1['Latitude'][i]) for i in range(len(frame1))]
point2=[Point(lon[i],lat[i]) for i in range(len(lat))]
frame2 = gpd.GeoDataFrame({"name": name,"geometry":point2},crs={'init' :'epsg:4326'})
#change the crs into london crs
frame2 = frame2.to_crs(london.crs)

In [11]:
#intersect to find common field
stations_near_kings = gpd.sjoin(frame2, kings_buffer, how="inner",op='intersects')

In [12]:
stations_near_kings

,geometry,name_left,index_right,name_right
50,POINT (531016.6251926103 180821.6366513587),Temple,0,KCL


In [13]:
station_names_kings=stations_near_kings[['name_left']]
station_names_kings.columns=['Name']
station_names_kings

,Name
50,Temple


In [14]:
#we get the station names closed to kings, now we combine the frame1 
#(the dataframe contains all the stations geographic informarion) with she station names closed with kings
newframe=pd.merge(station_names_kings, frame1, on='Name')

In [15]:
# we add a new column distance which can be used to store the distance between kings with each station
distance_data=[]
for i in range(len(newframe)):
    distance_data.append(round(distance((float(newframe.loc[i]['Longitude']),
                                         float(newframe.loc[i]['Latitude'])),kcl_coords).km,4))
newframe['distance']=distance_data

In [16]:
#we choose the closest station
chosen_station=newframe.loc[newframe['distance'].idxmin()]
kcl_chosen_station=chosen_station['Name']
print kcl_chosen_station

Temple


In [17]:
geolocator = Nominatim()
location = geolocator.geocode("WC1E 6BT")
print location

WC1E 6BT, United Kingdom


In [20]:
newframe#ucl

,Name,Latitude,Longitude,Line,distance
0,Euston Square,51.52584,-0.1357,"Circle, Hammersmith & City, Metropolitan",0.1664
1,Euston,51.52774,-0.13303,Northern,0.3927
2,Goodge Street,51.5206,-0.13441,Northern,0.4542
3,Warren Street,51.5245,-0.1381,"Northern, Victoria",0.3687


In [21]:
chosen_station=newframe.loc[newframe['distance'].idxmin()]
ucl_chosen_station=chosen_station['Name']
print ucl_chosen_station

Euston Square


def starting_point(place_name):

starting_point('WC2R 2LS')

destination('WC1E 6BT')

nx.shortest_path(tubeZone1,starting_point('WC2R 2LS'),destination('WC1E 6BT'))

In [23]:
naive_citymapper('WC2R 2LS','WC1E 6BT')

From WC2R 2LS to WC1E 6BT.
First, walk 0.27km to Temple station.
Pass through 7 stations, terminated at Euston Square station.    
And then walk 0.17km to destination. 
The tube path is 
Temple-->Embankment-->Westminster-->Green Park-->Bond Street-->Baker Street-->Great Portland Street-->Euston Square
